#Getting sentiment for userText

In [8]:
import sys
sys.path.append('/Users/alexsutherland/Dropbox/Insight/WeightLossReddit/')
import os
os.chdir('/Users/alexsutherland/Dropbox/Insight/WeightLossReddit/')
import redditDataIO
import pandas as pd
import qgrid

Getting the data:

In [21]:
combinedData = redditDataIO.loadData(tableName='combinedSentiment')
#combinedData = combinedData.drop(['index','index_x','index_y'], axis=1)

In [22]:
combinedData.describe()

,index,num_comments,endWeight,created_utc,height,score,weightChange,over_18,timeElapsed,startWeight,age,currentBMI,previousBMI,timeElapsedEpoch,timeElapsedMonths,weightChangeRate,fitnessCommentCount,totalCommentCount,fitnessCommentFraction,sentimentScores
count,3998.000000,3998.000000,3998.000000,3.998000e+03,3998.000000,3998.000000,3998.000000,3998.000000,3998.000000,3998.000000,3998.000000,3998.000000,3998.000000,3.998000e+03,3998.000000,3998.000000,3998.000000,3998.000000,3998.000000,3998.000000
mean,1998.500000,19.142821,182.920860,1.408113e+09,68.635318,306.120560,-52.637219,0.069785,6.575148,235.558079,24.701601,27.845442,35.791156,3.268023e+07,12.608110,-6.616578,45.584542,289.719360,0.308595,0.683092
std,1154.267517,33.650857,41.617924,1.698566e+07,4.062752,524.167141,37.666359,0.254816,7.578113,62.882969,5.198704,5.410668,8.205786,3.773805e+07,14.559429,4.624224,97.899280,334.041749,0.332802,1.930068
min,0.000000,0.000000,97.000000,1.363650e+09,58.000000,0.000000,-385.000000,0.000000,0.830000,108.000000,12.000000,17.088800,19.996200,6.048000e+05,0.233333,-30.000000,0.000000,1.000000,0.000000,-8.000000
25%,999.250000,5.000000,155.000000,1.395370e+09,66.000000,72.000000,-70.000000,0.000000,2.000000,190.000000,21.000000,24.196600,29.920675,1.040000e+07,4.012350,-8.925000,4.000000,32.000000,0.043336,0.000000
50%,1998.500000,10.000000,177.000000,1.409240e+09,69.000000,131.000000,-44.000000,0.000000,5.000000,225.000000,24.000000,26.625700,34.383700,2.080000e+07,8.024690,-5.696700,14.000000,135.000000,0.159579,0.000000
75%,2997.750000,19.000000,203.000000,1.423195e+09,72.000000,279.000000,-25.000000,0.000000,8.000000,270.000000,27.000000,30.333700,39.992100,3.380000e+07,13.040100,-3.240000,41.000000,443.000000,0.500000,2.000000
max,3997.000000,698.000000,449.000000,1.434440e+09,84.000000,5569.000000,-5.000000,1.000000,191.000000,720.000000,64.000000,59.583400,94.667600,5.670720e+08,218.778000,-0.037398,978.000000,1000.000000,1.000000,15.000000


Let's isolate the user text data:

In [33]:
userText = combinedData['userText']
userText

0     , 202 to 187. Planning on losing a few more, t...
1     . Final progress pics. Girlfriend said I was g...
2      later, got fat again! My girlfriend broke up ...
3     s doing C25K and completely changing my eating...
4                             s) and it's not over yet!
5     s) I haven't been this small in 13 years. Stil...
6               s) Even I can't believe the difference!
7                       s) Half the woman I used to be!
8               s) Yeah, I used to be more of a tool...
9                              s)Time to start lifting.
10    , goal 120-125. Feeling unmotivated right now ...
11     to lose the latest 5 pounds, but I'm not givi...
12                                 of Marathon Training
13    s so far) Calories in vs. calories out, are th...
14                              s) Time to start toning
...
3983    s) I lost most of the weight August-January an...
3984               and still going) I can finally run now
3985     High school graduation vs. Univ

Let's now get the sentiment dictionary:

In [9]:
def loadSentiments():
    afinnfile = open('Reference/AFINN-111.txt')
    scores = {} # initialize an empty dictionary
    for line in afinnfile:
      term, score  = line.split("\t")  # The file is tab-delimited. "\t" means "tab character"
      scores[term] = int(score)  # Convert the score to an integer.
    return scores

sentimentDictionary = loadSentiments()
sentimentDictionary

{'limited': -1,
 'suicidal': -2,
 'pardon': 2,
 'desirable': 2,
 'protest': -2,
 'lurking': -1,
 'controversial': -2,
 'hating': -3,
 'ridiculous': -3,
 'hate': -3,
 'aggression': -2,
 'increase': 1,
 'regretted': -2,
 'violate': -2,
 'granting': 1,
 'attracted': 1,
 'poorest': -2,
 'scold': -2,
 'bailout': -2,
 'sorry': -1,
 'regrets': -2,
 'struck': -1,
 'misreporting': -2,
 'vociferous': -1,
 'lurk': -1,
 'misunderstanding': -2,
 'distort': -2,
 'stolen': -2,
 'gratification': 2,
 'uncertain': -1,
 'stabbed': -2,
 'screaming': -2,
 'courageous': 2,
 'disturb': -2,
 'exaggerate': -2,
 'harried': -2,
 'solution': 1,
 'nigger': -5,
 'pardons': 2,
 'quaking': -2,
 'monopolized': -2,
 'censors': -2,
 'triumph': 4,
 'enjoy': 2,
 'shithead': -4,
 'tired': -2,
 'warns': -2,
 'landmark': 2,
 'elegant': 2,
 'fabulous': 4,
 'rigorous': 3,
 'emptiness': -1,
 'loathing': -3,
 'errors': -2,
 'hide': -1,
 'wreck': -2,
 'desirous': 2,
 'integrity': 2,
 'beaten': -2,
 'jocular': 2,
 'poison': -2,
 '

We need to split the comment text into just words:

In [10]:
import re
wordPattern = re.compile(r'(\w+)', re.IGNORECASE)

In [11]:
def calculateSentimentScore(commentText):
    splitCommentText = pd.Series(wordPattern.findall(commentText))
    return splitCommentText.ix[splitCommentText.isin(sentimentDictionary)].map(lambda x: sentimentDictionary[x]).sum()

In [29]:
wordList = pd.Series(['happy','horrible','mdf'])
wordList.ix[wordList.isin(sentimentDictionary)].map(lambda x: sentimentDictionary[x])

0    3
1   -3
dtype: int64

In [54]:
sentimentScores = userText.map(calculateSentimentScore)
combinedData['sentimentScores'] = sentimentScores
qgrid.show_grid(combinedData[['userText','sentimentScores']])

In [55]:
redditDataIO.exportDataToSQL(combinedData, tableName='combinedSentiment')

Index([u'permalink', u'userText', u'name', u'author', u'url', u'num_comments', u'endWeight', u'title', u'created_utc', u'height', u'weightUnit', u'score', u'weightChange', u'over_18', u'timeElapsed', u'gender', u'timeUnit', u'startWeight', u'age', u'currentBMI', u'previousBMI', u'timeElapsedEpoch', u'timeElapsedMonths', u'weightChangeRate', u'username', u'fitnessCommentCount', u'totalCommentCount', u'fitnessCommentFraction', u'sentimentScores'], dtype='object')
Exported Data to SQL Table (Name: combinedSentiment)


/Users/alexsutherland/anaconda/lib/python2.7/site-packages/pymysql/cursors.py:134: Warning: Unknown table 'redditweightdatabase.combinedsentiment'
  result = self._query(query)


In [56]:
combinedData.to_csv('Data/combinedSentiment.csv')

Plotting sentiment of high vs. low:

##Getting sentiment of user comments:

Let's calculate the sentiment for the user comments and append this onto the SQL table

In [12]:
userCommentData = redditDataIO.loadData('userCommentData')

In [13]:
userCommentData.head()

,index,score,controversiality,body,subreddit,link_title,link_id,created_utc,username,userid
0,0,2,0,Lenny is the man. I'm good friends with his so...,funny,Millionaires In America (Not OC),t3_2u8mg2,1422670000,BrovietUnion1,1
1,1,11,0,"CondoInternet, brobeans.",technology,SpaceX confirms $1 billion funding round with ...,t3_2t3giy,1421820000,BrovietUnion1,1
2,2,2,0,"Baller, thanks!! Happy new year!",Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420080000,BrovietUnion1,1
3,3,1,0,"Awesome, thanks for the advice!",Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420050000,BrovietUnion1,1
4,4,3,0,Great call! I'm from a place with 0 public tra...,Seattle,"First New Year's Eve in Seattle, and I have a ...",t3_2qwaqb,1420010000,BrovietUnion1,1


In [18]:
sentimentScores = userCommentData['body'].map(calculateSentimentScore)
userCommentData['sentimentScores'] = sentimentScores
#qgrid.show_grid(userCommentData.head(200))

Let's export this to the SQL database

In [19]:
redditDataIO.exportDataToSQL(userCommentData, 'userCommentData_Sentiment')

Index([u'index', u'score', u'controversiality', u'body', u'subreddit', u'link_title', u'link_id', u'created_utc', u'username', u'userid', u'sentimentScores'], dtype='object')
Exported Data to SQL Table (Name: userCommentData_Sentiment)


/Users/alexsutherland/anaconda/lib/python2.7/site-packages/pymysql/cursors.py:134: Warning: Unknown table 'redditweightdatabase.usercommentdata_sentiment'
  result = self._query(query)
